In [1]:
from keras.models import Sequential
model = Sequential()

Using Theano backend.


Using gpu device 0: GeForce GTX 970 (CNMeM is disabled)


In [2]:
from keras.layers.core import Dense, Activation
model.add(Dense(output_dim=64, input_dim=100, init='glorot_uniform'))
model.add(Activation("relu"))
model.add(Dense(output_dim=10, init='glorot_uniform'))
model.add(Activation("softmax"))

In [1]:
import pandas as pd
digits = pd.read_csv("/home/alpha/ML/data/mninst/train.csv")

In [2]:
digits.shape

(42000, 785)

In [3]:
train = digits.values[0:35000]
test = digits.values[35000:]

In [4]:
X_train = train[:,1:]
y_train = train[:,0]
X_test = test[:,1:]
y_test = test[:,0]

In [5]:
y_train

array([1, 0, 1, ..., 7, 7, 0])

In [6]:
'''Train a simple convnet on the MNIST dataset.
Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_cnn.py
Get to 99.25% test accuracy after 12 epochs (there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# the data, shuffled and split between tran and test sets
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta')

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Using Theano backend.
X_train shape: (35000, 1, 28, 28)
35000 train samples
7000 test samples
Train on 35000 samples, validate on 7000 samples
Epoch 1/12
35000/35000 [==============================] - 11s - loss: 0.3373 - acc: 0.8954 - val_loss: 0.0880 - val_acc: 0.9744
Epoch 2/12
35000/35000 [==============================] - 11s - loss: 0.1167 - acc: 0.9653 - val_loss: 0.0543 - val_acc: 0.9819
Epoch 3/12
35000/35000 [==============================] - 11s - loss: 0.0828 - acc: 0.9748 - val_loss: 0.0667 - val_acc: 0.9794
Epoch 4/12
35000/35000 [==============================] - 11s - loss: 0.0675 - acc: 0.9795 - val_loss: 0.0637 - val_acc: 0.9807
Epoch 5/12
35000/35000 [==============================] - 11s - loss: 0.0537 - acc: 0.9836 - val_loss: 0.0388 - val_acc: 0.9879
Epoch 6/12
35000/35000 [==============================] - 11s - loss: 0.0487 - acc: 0.9852 - val_loss: 0.0405 - val_acc: 0.9884
Epoch 7/12
35000/35000 [==============================] - 11s - loss: 0.0433 - acc: 0.986

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled)
